### Filter Sky Brightness Data
- Only keep daily max, non-zero min, non-zero mean, of King's Park Meteorological Station, Hong Kong Observatory
- From 1-1-2022 to 31-10-2024
- Miss August 2022, that month has no data from King's Park Meteorological Station, Hong Kong Observatory

In [1]:
import pandas as pd
import os

directory = r"C:\Users\cxoox\Desktop\sky_brightness"
output_dir = r"C:\Users\cxoox\Desktop\filtered_data"

os.makedirs(output_dir, exist_ok=True)
filtered_kings_park = pd.DataFrame()

if not os.path.exists(directory):
    print(f"Directory does not exist: {directory}")
else:
    for file in os.listdir(directory):
        if not file.endswith('.csv'):
            continue
 
        file_path = os.path.join(directory, file)
        print(f"Processing {file_path}")
        
        try:
            df = pd.read_csv(file_path)
            df = df[df['device_code'].isin(['KP'])]  # Filter device
            
            # Parse datetime with multiple format attempts
            for fmt in ['%d/%m/%Y %H:%M:%S', '%m/%d/%Y %H:%M:%S', '%d/%m/%y %H:%M:%S', '%Y-%m-%d %H:%M:%S']:
                try:
                    df['received_adjusted'] = pd.to_datetime(df['received_adjusted'], format=fmt)
                    break
                except:
                    continue
            
            if not pd.api.types.is_datetime64_any_dtype(df['received_adjusted']):
                print(f"Datetime parsing failed in {file}")
                continue
                
            # Sort by timestamp
            df = df.sort_values('received_adjusted')
            
            # Create hour-level grouping key
            df['hour_group'] = df['received_adjusted'].dt.strftime('%Y-%m-%d %H:00')
            
            # Keep first observation in each hour group
            df = df.drop_duplicates(subset=['hour_group'], keep='first')
            
            filtered_kings_park = pd.concat([filtered_kings_park, df], ignore_index=True)
            
        except Exception as e:
            print(f"Error processing {file}: {str(e)}")

# Final cleanup
filtered_kings_park = filtered_kings_park.drop(columns=['hour_group'])  # Remove helper column
filtered_kings_park = filtered_kings_park.drop_duplicates()  # Safety check

# Extract date from timestamp
filtered_kings_park['date'] = filtered_kings_park['received_adjusted'].dt.date

# Calculate daily statistics
def non_zero_mean(x):
    non_zero = x[x > 0]
    return non_zero.mean() if len(non_zero) > 0 else None

daily_stats = filtered_kings_park.groupby('date')['nsb'].agg([
    ('Max Night Sky Brightness (MPSAS)', 'max'),
    ('Min Night Sky Brightness (Non-zero) (MPSAS)', lambda x: x[x > 0].min() if any(x > 0) else None),
    ('Mean Night Sky Brightness (Excluded zero) (MPSAS)', non_zero_mean)  # Mean excluding zero values
]).reset_index()

# Save results
output_path = os.path.join(output_dir, "daily_nsb.csv")
daily_stats.to_csv(output_path, index=False)
print(f"Saved daily statistics for {len(daily_stats)} days to {output_path}")
print("\nSample of daily statistics (excluding zero values from mean calculation):")
print(daily_stats.head())

Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_01.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_02.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_03.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_04.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_05.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_06.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_07.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_08.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_09.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_10.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_11.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_12.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2023_01.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2023_02.csv
Processing C:\Users\cxoox\Desktop\sky_brightness

In [2]:
import pandas as pd

weather_data = ["Daily Maximum Hong Kong Heat Index", 
                "Daily Mean Hong Kong Heat Index", 
                "Daily Mean Wet Bulb Temperature (°C)",
                'Daily Mean Dew Point Temperature (°C)',
                "Daily Mean Amount of Cloud",
                "Daily Mean Pressure (hPa)",
                "Daily Total Evaporation (mm)",
                "Daily Total Rainfall (mm)",
                "Daily Mean Relative Humidity (%)",
                "Daily Maximum Temperature (°C)",
                "Daily Minimum Temperature (°C)",
                "Daily Mean Temperature (°C)",
                "Daily Global Solar Radiation (MJ/m2)",
                "Daily Total Bright Sunshine (hours)",
                "Daily Mean Wind Speed (m/s)"]

weather_data_path = [r"C:\Users\cxoox\Desktop\weather\daily_KP_MEANHKHI_ALL.csv",
                     r"C:\Users\cxoox\Desktop\weather\daily_KP_MAXHKHI_ALL.csv", 
                     r"C:\Users\cxoox\Desktop\weather\daily_KP_WET_ALL.csv",
                     r"C:\Users\cxoox\Desktop\weather\daily_KP_DEW_ALL.csv",
                     r"C:\Users\cxoox\Desktop\weather\daily_HKO_CLD_ALL.csv",
                     r"C:\Users\cxoox\Desktop\weather\daily_HKO_MSLP_ALL.csv",
                     r"C:\Users\cxoox\Desktop\weather\daily_KP_EVAP_ALL.csv",
                     r"C:\Users\cxoox\Desktop\weather\daily_KP_RF_ALL.csv",
                     r"C:\Users\cxoox\Desktop\weather\daily_KP_RH_ALL.csv",
                     r"C:\Users\cxoox\Desktop\weather\CLMMAXT_KP_.csv",
                     r"C:\Users\cxoox\Desktop\weather\CLMMINT_KP_.csv",
                     r"C:\Users\cxoox\Desktop\weather\CLMTEMP_KP_.csv",
                     r"C:\Users\cxoox\Desktop\weather\daily_KP_GSR_ALL.csv",
                     r"C:\Users\cxoox\Desktop\weather\daily_KP_SUN_ALL.csv",
                     r"C:\Users\cxoox\Desktop\weather\daily_KP_WSPD_ALL.csv"]


# 1. Load the combined filtered data
filtered_kings_park = pd.read_csv(r"C:\Users\cxoox\Desktop\filtered_data\daily_nsb.csv")

# Convert the 'received_adjusted' column to a date column, handling invalid values
filtered_kings_park['date'] = pd.to_datetime(
    filtered_kings_park['date'],
    errors='coerce'  # Convert invalid dates to NaT
).dt.date

# Drop rows with invalid dates (if any)
filtered_kings_park = filtered_kings_park.dropna(subset=['date'])

# 2. Load the daily mean cloud cover data
for file_path in weather_data_path:
    if not os.path.exists(file_path):
        print(f"File does not exist: {file_path}")
        continue

    # Read the CSV file, skipping the first two rows and using only the relevant columns
    df = pd.read_csv(
        file_path,
        skiprows=2,
        header=0,
        usecols=["年/Year", "月/Month", "日/Day", "數值/Value"]
    )

    # Rename columns for easier handling
    df = df.rename(columns={
        "年/Year": "Year",
        "月/Month": "Month",
        "日/Day": "Day",
        "數值/Value": weather_data[weather_data_path.index(file_path)]
    })

    # Convert Year, Month, and Day to integers, handling invalid values
    for col in ["Year", "Month", "Day"]:
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')  # Int64 supports NaN

    # Combine Year, Month, and Day into a single date column, handling invalid values
    df['date'] = pd.to_datetime(
        df[["Year", "Month", "Day"]].astype(str).agg("-".join, axis=1),
        format="%Y-%m-%d",
        errors='coerce'  # Convert invalid dates to NaT
    ).dt.date

    # Drop rows with invalid dates (if any)
    df = df.dropna(subset=['date'])
    
    filtered_kings_park = pd.merge(
        filtered_kings_park,
        df[['date', weather_data[weather_data_path.index(file_path)]]],
        on='date',
        how='left'
    )
    

# 5. Save the merged data
output_file_path = r"C:\Users\cxoox\Desktop\filtered_data\nsb_weather_merged.csv"
filtered_kings_park.to_csv(output_file_path, index=False)
print(f"Merge completed! Data saved to {output_file_path}")
print("First 5 rows of the merged data:")
print(filtered_kings_park.head())

Merge completed! Data saved to C:\Users\cxoox\Desktop\filtered_data\nsb_weather_merged.csv
First 5 rows of the merged data:
         date  Max Night Sky Brightness (MPSAS)  \
0  2022-01-01                             16.71   
1  2022-01-02                             17.49   
2  2022-01-03                             17.54   
3  2022-01-04                             17.74   
4  2022-01-05                             16.87   

   Min Night Sky Brightness (Non-zero) (MPSAS)  \
0                                         9.03   
1                                         7.82   
2                                         7.87   
3                                         8.02   
4                                         8.11   

   Mean Night Sky Brightness (Excluded zero) (MPSAS)  \
0                                          15.205000   
1                                          15.688571   
2                                          15.614286   
3                                          1

### Combine sun and moon files respectively

In [3]:
# Combine csv files into one
import pandas as pd
import os

combined = pd.DataFrame()
directory = r"C:\Users\cxoox\Desktop\weather\sun"

for file in os.listdir(directory):
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path)
    combined = pd.concat([combined, df])
    
combined.to_csv(r"C:\Users\cxoox\Desktop\filtered_data\combined_sun.csv", index=False)
print("Combined sun CSV files into one.")

combined = pd.DataFrame()
directory = r"C:\Users\cxoox\Desktop\weather\moon"

for file in os.listdir(directory):
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path)
    combined = pd.concat([combined, df])
    
combined.to_csv(r"C:\Users\cxoox\Desktop\filtered_data\combined_moon.csv", index=False)
print("Combined moon CSV files into one.")
    

Combined sun CSV files into one.
Combined moon CSV files into one.


### Merge sun and moon data into nsb_weather dataset

In [4]:
import pandas as pd

weather_data = ["Sun", "Moon"]

weather_data_path = [r"C:\Users\cxoox\Desktop\filtered_data\combined_sun.csv", r"C:\Users\cxoox\Desktop\filtered_data\combined_moon.csv"]


# 1. Load the combined filtered data
filtered_kings_park = pd.read_csv(r"C:\Users\cxoox\Desktop\filtered_data\nsb_weather_merged.csv")

filtered_kings_park = filtered_kings_park.rename(columns={"date": "Date"})

# Convert the 'received_adjusted' column to a date column, handling invalid values
filtered_kings_park['Date'] = pd.to_datetime(
    filtered_kings_park['Date'],
    errors='coerce'  # Convert invalid dates to NaT
).dt.date

# Drop rows with invalid dates (if any)
filtered_kings_park = filtered_kings_park.dropna(subset=['Date'])

# 2. Load the daily mean cloud cover data
for file_path in weather_data_path:
    if not os.path.exists(file_path):
        print(f"File does not exist: {file_path}")
        continue

    # Read the CSV file, skipping the first two rows and using only the relevant columns
    df = pd.read_csv(
        file_path,
        skiprows=0,
        header=0,
        usecols=["YYYY-MM-DD", "RISE", "TRAN.", "SET"]
    )

    # Rename columns for easier handling
    df = df.rename(columns={
        "YYYY-MM-DD": "Date",
        "RISE": f"{weather_data[weather_data_path.index(file_path)]} Rise",
        "TRAN.": f"{weather_data[weather_data_path.index(file_path)]} Transit",
        "SET": f"{weather_data[weather_data_path.index(file_path)]} Set"
    })


    # Combine Year, Month, and Day into a single date column, handling invalid values
    df['Date'] = pd.to_datetime(
        df["Date"],
        format="%Y-%m-%d",
        errors='coerce'  # Convert invalid dates to NaT
    ).dt.date

    # Drop rows with invalid dates (if any)
    df = df.dropna(subset=['Date'])

    
    filtered_kings_park = pd.merge(
        filtered_kings_park,
        df[['Date', f"{weather_data[weather_data_path.index(file_path)]} Rise", f"{weather_data[weather_data_path.index(file_path)]} Transit", f"{weather_data[weather_data_path.index(file_path)]} Set"]],
        on='Date',
        how='left'
    )
    

# 5. Save the merged data
output_file_path = r"C:\Users\cxoox\Desktop\filtered_data\nsb_weather_merged.csv"
filtered_kings_park.to_csv(output_file_path, index=False)
print(f"Merge completed! Data saved to {output_file_path}")
print("First 5 rows of the merged data:")
print(filtered_kings_park.head())

Merge completed! Data saved to C:\Users\cxoox\Desktop\filtered_data\nsb_weather_merged.csv
First 5 rows of the merged data:
         Date  Max Night Sky Brightness (MPSAS)  \
0  2022-01-01                             16.71   
1  2022-01-02                             17.49   
2  2022-01-03                             17.54   
3  2022-01-04                             17.74   
4  2022-01-05                             16.87   

   Min Night Sky Brightness (Non-zero) (MPSAS)  \
0                                         9.03   
1                                         7.82   
2                                         7.87   
3                                         8.02   
4                                         8.11   

   Mean Night Sky Brightness (Excluded zero) (MPSAS)  \
0                                          15.205000   
1                                          15.688571   
2                                          15.614286   
3                                          1

In [ ]:
import pandas as pd
import os

directory = r"C:\Users\cxoox\Desktop\sky_brightness"
output_dir = r"C:\Users\cxoox\Desktop\filtered_data"

os.makedirs(output_dir, exist_ok=True)
filtered_kings_park = pd.DataFrame()

if not os.path.exists(directory):
    print(f"Directory does not exist: {directory}")
else:
    for file in os.listdir(directory):
        if not file.endswith('.csv'):
            continue
 
        file_path = os.path.join(directory, file)
        print(f"Processing {file_path}")
        
        try:
            df = pd.read_csv(file_path)
            df = df[df['device_code'].isin(['AP'])]  # Filter device
            
            # Parse datetime with multiple format attempts
            for fmt in ['%d/%m/%Y %H:%M:%S', '%m/%d/%Y %H:%M:%S', '%d/%m/%y %H:%M:%S', '%Y-%m-%d %H:%M:%S']:
                try:
                    df['received_adjusted'] = pd.to_datetime(df['received_adjusted'], format=fmt)
                    break
                except:
                    continue
            
            if not pd.api.types.is_datetime64_any_dtype(df['received_adjusted']):
                print(f"Datetime parsing failed in {file}")
                continue
                
            # Sort by timestamp
            df = df.sort_values('received_adjusted')
            
            # Create hour-level grouping key
            df['hour_group'] = df['received_adjusted'].dt.strftime('%Y-%m-%d %H:00')
            
            # Keep first observation in each hour group
            df = df.drop_duplicates(subset=['hour_group'], keep='first')
            
            filtered_kings_park = pd.concat([filtered_kings_park, df], ignore_index=True)
            
        except Exception as e:
            print(f"Error processing {file}: {str(e)}")

# Final cleanup
filtered_kings_park = filtered_kings_park.drop(columns=['hour_group'])  # Remove helper column
filtered_kings_park = filtered_kings_park.drop_duplicates()  # Safety check

# Extract date from timestamp
filtered_kings_park['date'] = filtered_kings_park['received_adjusted'].dt.date

# Calculate daily statistics
def non_zero_mean(x):
    non_zero = x[x > 0]
    return non_zero.mean() if len(non_zero) > 0 else None

daily_stats = filtered_kings_park.groupby('date')['nsb'].agg([
    ('Max Night Sky Brightness (MPSAS)', 'max'),
    ('Min Night Sky Brightness (Non-zero) (MPSAS)', lambda x: x[x > 0].min() if any(x > 0) else None),
    ('Mean Night Sky Brightness (Excluded zero) (MPSAS)', non_zero_mean)  # Mean excluding zero values
]).reset_index()

# Save results
output_path = os.path.join(output_dir, "daily_nsb_AP.csv")
daily_stats.to_csv(output_path, index=False)
print(f"Saved daily statistics for {len(daily_stats)} days to {output_path}")
print("\nSample of daily statistics (excluding zero values from mean calculation):")
print(daily_stats.head())

Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_01.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_02.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_03.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_04.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_05.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_06.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_07.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_08.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_09.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_10.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_11.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2022_12.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2023_01.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2023_02.csv
Processing C:\Users\cxoox\Desktop\sky_brightness

In [20]:
import pandas as pd
import numpy as np

# 读取数据
data_AP = pd.read_csv(r"C:\Users\cxoox\Desktop\filtered_data\daily_nsb_AP.csv")
data_merged = pd.read_csv(r"C:\Users\cxoox\Desktop\filtered_data\nsb_weather_merged.csv")

# 重命名列以统一日期字段
data_AP = data_AP.rename(columns={"date": "Date"})

# 计算 L_viirs（示例值，需根据实际数据校准）
viiirs_radiation = 94.262375  # 需验证此值是否合理
L_viirs = 20.723 - 2.5 * np.log10(viiirs_radiation)

# 按日期合并数据，仅保留匹配的行
merged = pd.merge(data_merged, data_AP, on="Date", how="inner")

# 修正夜空亮度值（假设逻辑正确）
brightness_columns = [
    "Mean Night Sky Brightness (Excluded zero) (MPSAS)",
    "Min Night Sky Brightness (Non-zero) (MPSAS)",
    "Max Night Sky Brightness (MPSAS)",
]


for col in brightness_columns:
    merged[col] = merged[col + "_x"] + (merged[col + "_y"] - L_viirs)  # set to 0 if negative
    merged[col] = merged[col].clip(lower=0)  # Clip to ensure no negative values

# 计算新的夜空亮度值（示例值，需根据实际数据校准）

# 删除冗余列
merged = merged.drop(columns=[col + "_x" for col in brightness_columns] + [col + "_y" for col in brightness_columns])

# Move the three columns to the front right after the date column
for col in brightness_columns:
    merged.insert(1, col, merged.pop(col))

# 保存结果
merged.to_csv(r"C:\Users\cxoox\Desktop\filtered_data\nsb_weather_merged_corrected.csv", index=False)
print("Night sky brightness values adjusted successfully.")
merged.head()

Night sky brightness values adjusted successfully.


,Date,Max Night Sky Brightness (MPSAS),Min Night Sky Brightness (Non-zero) (MPSAS),Mean Night Sky Brightness (Excluded zero) (MPSAS),Daily Maximum Hong Kong Heat Index,Daily Mean Hong Kong Heat Index,Daily Mean Wet Bulb Temperature (°C),Daily Mean Dew Point Temperature (°C),Daily Mean Amount of Cloud,Daily Mean Pressure (hPa),...,Daily Mean Temperature (°C),Daily Global Solar Radiation (MJ/m2),Daily Total Bright Sunshine (hours),Daily Mean Wind Speed (m/s),Sun Rise,Sun Transit,Sun Set,Moon Rise,Moon Transit,Moon Set
0,2022-01-01,21.102846,2.782846,17.656179,15.4,17.0,14.2,12.2,68.0,1024.4,...,16.7,7.30,1.6,7.9,07:03,12:27,17:51,05:12,10:44,16:14
1,2022-01-02,22.422846,0.172846,18.495703,17.2,19.6,15.0,12.8,10.0,1022.5,...,17.8,15.52,9.5,5.4,07:03,12:27,17:51,06:22,11:50,17:17
2,2022-01-03,22.272846,0.212846,18.294989,16.9,18.7,15.2,13.5,27.0,1021.1,...,17.5,15.15,8.0,9.6,07:03,12:28,17:52,07:29,12:56,18:24
3,2022-01-04,22.492846,0.512846,17.924275,17.1,19.1,15.5,13.1,49.0,1019.6,...,18.6,16.46,8.7,11.5,07:04,12:28,17:53,08:30,14:00,19:33
4,2022-01-05,21.422846,0.692846,16.968560,19.1,21.4,16.9,14.7,50.0,1017.3,...,20.1,16.78,8.8,8.2,07:04,12:29,17:53,09:23,14:59,20:39
